In [1]:
import pandas as pd
import GladUtil as gd

## IBSS와 GOMS수수료 산출결과를 비교함

In [163]:
path ='D:/글로벌백업(180329)/수수료비교(goms-ibss)/'
in_file = '202105_지급수수료산출_손해보험_FC수수료산출.xlsx'
gb = '장기'
df = pd.read_excel(path+in_file,header=0,dtype={'계약일자':object})
df_g = df[df['구분']==gb]

path ='D:/글로벌백업(180329)/04.수수료작업/202104수수료/'
in_file = '마감실적리스트(장기)-202105261605.xlsx'
df = pd.read_excel(path+in_file,header=0)
df_i = df[df['계약일']>='2021-04-01']

In [164]:
df_i_grp = df_i.groupby(['증권번호'])['수수료S','수정보험료'].agg(sum)
df_i_grp['지급율'] = df_i_grp['수수료S'] / df_i_grp['수정보험료'] * 100
df_g_grp = df_g.groupby(['증권번호'])['산출금액','원수사성적'].agg(sum)
df_g_grp['지급율'] = df_g_grp['산출금액'] / df_g_grp['원수사성적'] * 100

D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  """Entry point for launching an IPython kernel.
D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  This is separate from the ipykernel package so we can avoid doing imports until


In [165]:
df_ply = pd.merge(df_i_grp,df_g_grp,how='outer',on=['증권번호'],indicator=True).reset_index().astype({"_merge": str}).fillna(0)
df_ply['차이'] = df_ply['수수료S'] - df_ply['산출금액']

In [177]:
# ibss로 부터 데이터가져오기
df_temp = df_i.drop_duplicates(['증권번호'], keep='first')  #증번중복은 첫번째 건만 추출
df_unload = df_ply[df_ply['_merge'] != 'right_only'].join(df_temp.set_index(['증권번호'])[['지점','담당','보험회사','보종','계약일']],
                                                          on=(['증권번호']), rsuffix='g', how='left')
# goms로 부터 데이터가져오기
df_temp = df_g.drop_duplicates(['증권번호'], keep='first')
df_temp = df_ply[df_ply['_merge'] == 'right_only'].join(df_temp.set_index(['증권번호'])[['지역단','성명','보험사명','상품소분류','계약일자']],
                                                        on=(['증권번호']), rsuffix='g', how='left')
df_temp.columns = df_unload.columns

In [178]:
## 차이발생건만 list up
## left_only = ibss only,  right_only = goms only
df_unload = pd.concat([df_unload,df_temp])
df_unload[abs(df_unload['차이']) > 100].sort_values('증권번호')

,증권번호,수수료S,수정보험료,지급율_x,산출금액,원수사성적,지급율_y,_merge,차이,지점,담당,보험회사,보종,계약일
919,20204540401,0.0,0.0,0.000000,4984.0,3759.0,132.588454,right_only,-4984.0,글래드송내,이현자,KB손보,실손,20200824
920,20211479102,0.0,0.0,0.000000,5097.0,3844.0,132.596254,right_only,-5097.0,글래드송내,이선영,KB손보,실손,20210319
208,320211041404,13205.0,7056.0,187.145692,0.0,0.0,0.000000,left_only,13205.0,글래드라온,박종철,DB손보,실손,2021-04-25
249,42143459510000,2539.0,1464.0,173.428962,2403.0,1386.0,173.376623,both,136.0,글래드조은,김명주,흥국화재,실손,2021-04-12
345,52111633900000,27821.0,8903.0,312.490172,0.0,0.0,0.000000,left_only,27821.0,글래드의정부,최주영,삼성화재,보장,2021-04-20
921,61687-103946,0.0,0.0,0.000000,12073.0,6526.0,184.998468,right_only,-12073.0,글래드상동,이경화2,메리츠화재,실손,20200915
922,61687-124187,0.0,0.0,0.000000,12071.0,6525.0,184.996169,right_only,-12071.0,글래드상동,강미경,메리츠화재,실손,20201012
923,61687-135424,0.0,0.0,0.000000,12073.0,6526.0,184.998468,right_only,-12073.0,글래드상동,김순자,메리츠화재,실손,20201023
924,61715-2451,0.0,0.0,0.000000,1092949.0,524624.0,208.329966,right_only,-1092949.0,글래드상동,이경화2,메리츠화재,보장성,20200915
925,61715-3901,0.0,0.0,0.000000,292993.0,140639.0,208.329837,right_only,-292993.0,글래드상동,김순자,메리츠화재,보장성,20201023


### --------------------------------------------------------------------------------------------------------

In [175]:
df[df['증권번호']=='6AAAK-2535']
# df_i.dtypes

,본부,지점,팀,담당,계약일,보험회사,대리점,보험상품,상품코드,보종,...,계약자 생년월일/성별,영수일,증권번호,개시일,만기일,경로,기타1,납입방법,상담자,메모
